### 데이터 분석 기본 - 머신러닝/딥러닝 기본
 - 다이아몬드 데이터 셋 머신러닝 모델 구축

▶ 다이아몬드 데이터 셋 머신러닝 모델 구축
   - (1) poltly 데이터 활용 시각화
   - (2) 다이아몬드 품질 예측 모델 만들기(k-NN)
   - (3) 의사결정트리, 앙상블 모델 만들기 및 평가

In [10]:
# 라이브러리 불러오기
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd

### 데이터 탐색

In [11]:
# 데이터 불러오기
diamonds = sns.load_dataset("diamonds")
diamonds

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


* 데이터 셋 설명
 * Cut : GIA(Gemological Institute of America, 미국보석학연구원)에 의해 제정된 것으로 다이아몬드 등급 시스템 표준화되어 업계 전반에 걸쳐 사용되고 있음.
 * 5등급의 체계는 다이아몬드 제조업체, 유통업체, 보석 감정사 등 다이아몬드 산업 전반에 걸쳐 통용되는 국제 표준.
```
Ideal: 이 등급은 가능한 최고의 광채를 내기 위해 거의 완벽하게 자르고 연마된 다이아몬드를 말합니다. Ideal 컷은 빛의 반사와 굴절을 최대화하여 다이아몬드가 가진 가능한 최고의 빛을 발산하게 합니다.
Premium: Premium 등급의 다이아몬드는 매우 높은 수준의 광채와 반짝임을 제공하며, 이상적인 컷에 매우 가깝게 연마된 것으로 평가받습니다. 이 등급은 또한 뛰어난 외관을 제공하면서도 가격면에서 좀 더 접근하기 쉬운 선택지를 제공할 수 있습니다.
Very Good: Very Good 등급은 높은 수준의 광채와 반짝임을 유지하면서도, Ideal 등급보다는 약간의 타협을 볼 수 있는 다이아몬드입니다. 이 컷은 빛을 잘 반사하여 뛰어난 빛의 성능을 발휘하지만, Ideal 컷만큼은 아닙니다.
Good: Good 등급의 다이아몬드는 평균 이상의 성능을 나타내지만, 상위 등급의 다이아몬드에 비해 빛의 반사가 덜 최적화되어 있습니다. 이 등급은 여전히 좋은 광채와 반짝임을 제공하지만, 비용을 더 절감하고자 하는 구매자에게 적합할 수 있습니다.
Fair: Fair 등급은 다이아몬드가 기본적인 품질 기준을 충족하며, 일부 광채와 반짝임은 있지만 최상의 성능을 내지는 못합니다. 이 컷은 가격에 민감한 구매자에게는 매력적일 수 있으나, 더 높은 광채를 원하는 구매자에게는 덜 선호될 수 있습니다.
```

### Scatter Plot

In [13]:
# 캐럿 크기에 따른 다이아몬드의 가격 변동
# 산점도 그래프 - Figure 객체 생성. go.Scatter()
fig = go.Figure(data=go.Scatter(x=diamonds['carat'],
                                y=diamonds['price'],
                                mode='markers'))

# 그래프의 레이아웃을 설정(update_layout)
fig.update_layout(title='Price vs. Carat',
                  xaxis_title='Carat', yaxis_title='Price')

fig.show()

### Bar Plot

In [14]:
# 다이아몬드 컷별로 평균 가격이 어떻게 달라지는 쉽게 비교 분석
average_prices = diamonds.groupby('cut')['price'].mean()

# 막대그래프 - Figure객체 생성. go.Bar()
fig = go.Figure(data=go.Bar(x=average_prices.index,
                            y=average_prices))

# 그래프의 레이아웃 설정
fig.update_layout(title='Average Price by Cut',
                  xaxis_title='Cut', yaxis_title='Average Price')

fig.show()

### BoxPlot

In [15]:
# 다이아몬드의 가격 분포를 컷(Cut)별로 박스 플롯으로 시각화
# 박스 플롯은 최소값, 1사분위수, 중앙값, 3사분위수, 최대값과 함께 이상치 표시
fig = go.Figure()

for cut in diamonds['cut'].unique():
    fig.add_trace(go.Box(y=diamonds[diamonds['cut'] == cut]['price'], name=cut))

# 그래프의 레이아웃 설정
fig.update_layout(title='Price Distribution by Cut',
                  xaxis_title='Cut', yaxis_title='Price')

fig.show()

### Pie 그래프

In [16]:
diamonds['clarity'].unique()

['SI2', 'SI1', 'VS1', 'VS2', 'VVS2', 'VVS1', 'I1', 'IF']
Categories (8, object): ['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1']

In [17]:
# 다이아몬드의 클라리티(clarity) 등급 분포를 보여준다.
clarity_counts = diamonds['clarity'].value_counts()
fig = go.Figure(data=go.Pie(labels=clarity_counts.index,
                            values=clarity_counts))

# 그래프의 레이아웃 설정
fig.update_layout(title='Clarity Ratings', showlegend=True)
fig.show()

* 설명
```
IF (Internally Flawless): '내부적으로 완벽한' 등급으로, 10배 확대시 내부에 결함이 전혀 보이지 않습니다. 표면에는 극미한 결함이 있을 수 있지만, 매우 어렵게만 관찰됩니다.
VVS1, VVS2 (Very Very Slightly Included 1 and 2): '매우 매우 약간의 내포물'이 있는 등급으로, VVS1은 VVS2보다 내포물이 더 적거나 덜 가시적입니다. 이 결함들은 10배 확대시에도 전문가만이 겨우 식별할 수 있을 정도로 미세합니다.
VS1, VS2 (Very Slightly Included 1 and 2): '매우 약간의 내포물'이 있는 등급으로, VS1은 VS2보다 내포물이 더 적거나 덜 가시적입니다. 이 내포물들은 10배 확대시 전문가에 의해 쉽게 식별될 수 있지만, 육안으로는 잘 보이지 않습니다.
SI1, SI2 (Slightly Included 1 and 2): '약간의 내포물'이 있는 등급으로, SI1은 SI2보다 내포물이 더 적거나 덜 가시적입니다. 이 결함들은 10배 확대시 쉽게 식별될 수 있으며, 경우에 따라 육안으로도 볼 수 있습니다.
I1 (Included 1): '포함된' 등급으로, 이 등급의 다이아몬드는 내포물이 육안으로도 식별될 수 있을 정도로 명확하며, 이는 다이아몬드의 투명도와 광채에 영향을 미칠 수 있습니다.
```

In [18]:
# 다이아몬드의 가격, 캐럿, 깊이 간의 관계를 시각화하는 데 사용
fig = go.Figure(data=[go.Scatter3d(x=diamonds['carat'],
                                   y=diamonds['price'],
                                   z=diamonds['depth'],
                                   mode='markers', marker=dict(size=3,
                                   color=diamonds['depth'], colorscale='Viridis'))])

fig.update_layout(title='Price, Carat, and Depth',
                  scene=dict(xaxis_title='Carat',
                             yaxis_title='Price',
                             zaxis_title='Depth'))

fig.show()

### (2) 다이아몬드 품질 예측 모델 만들기(k-NN)

In [21]:
diamonds

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [22]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [24]:
# table,	price 의 컬럼을 활용한 cut 예측
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### 데이터 전처리 및 데이터 분할

In [25]:
# 필요한 컬럼 선택
X = diamonds[['table', 'price']]
y = diamonds['cut']

# 레이블 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, y

(       table  price
 26546   64.0  16231
 9159    60.0   4540
 14131   58.0   5729
 15757   65.0   6300
 24632   57.0  12968
 ...      ...    ...
 11284   59.0   4975
 44732   55.0   1617
 38158   58.0   1014
 860     59.0   2871
 15795   58.0   6320
 
 [43152 rows x 2 columns],
 array([2, 3, 1, ..., 4, 3, 2]))

### 모델 생성 및 모델 평가

In [26]:
# KNN 모델 생성
knn = KNeighborsClassifier(n_neighbors=5)

# 모델 학습
knn.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = knn.predict(X_test)

# 정확도 계산
accuracy = knn.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.52


### 의사결정트리 모델 구축 및 평가

In [29]:
from sklearn.tree import DecisionTreeClassifier

# 필요한 컬럼 선택
X = diamonds[['table', 'price']]
y = diamonds['cut']

# 레이블 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 의사결정트리 모델 생성
DTC_model = DecisionTreeClassifier(max_depth=7)

# 모델 학습
DTC_model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = DTC_model.predict(X_test)

# 정확도 계산
accuracy = DTC_model.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.62


### 앙상블(랜덤포레스트) 모델 구축 및 평가

In [30]:
from sklearn.ensemble import RandomForestClassifier

# 필요한 컬럼 선택
X = diamonds[['table', 'price']]
y = diamonds['cut']

# 레이블 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 앙상블 모델 생성
rf_model = RandomForestClassifier(max_depth=7)

# 모델 학습
rf_model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = rf_model.predict(X_test)

# 정확도 계산
accuracy = rf_model.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.63
